In [26]:
import functools
import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 1s 35us/step


In [3]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [6]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [8]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True,
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [10]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'male' b'male']
age                 : [28. 32. 22. 30. 47.]
n_siblings_spouses  : [1 0 0 0 0]
parch               : [2 0 0 0 0]
fare                : [23.45   7.925  7.125 13.     9.   ]
class               : [b'Third' b'Third' b'Third' b'Second' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'n' b'y' b'y' b'y' b'y']


In [11]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)



sex                 : [b'male' b'male' b'male' b'male' b'male']
age                 : [50. 34. 40. 28. 18.]
n_siblings_spouses  : [1 1 0 0 0]
parch               : [0 0 0 0 0]
fare                : [106.425  21.     31.     56.496  11.5  ]
class               : [b'First' b'Second' b'First' b'Third' b'Second']
deck                : [b'C' b'unknown' b'A' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Cherbourg' b'Southampton' b'Southampton']
alone               : [b'n' b'n' b'y' b'y' b'y']


In [12]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [28. 25. 11. 21. 31.]
n_siblings_spouses  : [0 1 4 2 0]
class               : [b'Second' b'Third' b'Third' b'First' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'B' b'unknown']
alone               : [b'y' b'n' b'n' b'n' b'y']


In [13]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path,
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [28. 38. 32. 47. 42.]
n_siblings_spouses  : [0. 0. 0. 0. 0.]
parch               : [0. 1. 0. 0. 0.]
fare                : [  7.729 153.462   8.363   7.25    7.55 ]


In [18]:
example_batch, labels_batch = next(iter(temp_dataset))

In [16]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label



In [17]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())



[[22.     0.     0.     7.75 ]
 [28.     1.     0.    15.5  ]
 [27.     0.     0.    26.   ]
 [ 4.     1.     1.    11.133]
 [28.     0.     0.     7.225]]

[1 0 0 1 0]


In [29]:
show_batch(raw_train_data)
example_batch, labels_batch = next(iter(temp_dataset))

sex                 : [b'male' b'female' b'male' b'male' b'female']
age                 : [28. 29. 28.  9. 28.]
n_siblings_spouses  : [0 1 0 1 0]
parch               : [0 1 0 1 0]
fare                : [ 7.225 10.462  7.775 15.9    7.229]
class               : [b'Third' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'G' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'y' b'n' b'y' b'n' b'y']


In [20]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [21]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [22]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'female' b'female' b'male']
class               : [b'Third' b'Third' b'Third' b'First' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'C' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Queenstown' b'Cherbourg' b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'y']
numeric             : [[ 28.      0.      0.      8.05 ]
 [  9.      1.      1.     15.9  ]
 [ 28.      0.      0.      7.75 ]
 [ 36.      0.      0.    135.633]
 [ 39.      0.      0.     13.   ]]


In [31]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [32]:
#Continuous data should always be normalized.

import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc


,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [33]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [34]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [35]:
#See what you just created
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)
numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7fd978eabcb0>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [36]:
example_batch['numeric']


<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[35.   ,  0.   ,  0.   , 21.   ],
       [21.   ,  0.   ,  0.   , 10.5  ],
       [18.   ,  1.   ,  1.   ,  7.854],
       [25.   ,  0.   ,  0.   ,  7.05 ],
       [ 3.   ,  1.   ,  1.   , 18.75 ]], dtype=float32)>

In [37]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[ 0.429, -0.474, -0.479, -0.245],
       [-0.69 , -0.474, -0.479, -0.437],
       [-0.93 ,  0.395,  0.782, -0.486],
       [-0.37 , -0.474, -0.479, -0.501],
       [-2.128,  0.395,  0.782, -0.286]], dtype=float32)

In [38]:
#Categorical data
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [39]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [40]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [41]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [42]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [43]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     1.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.429 -0.474
 -0.479 -0.245  0.     1.   ]


In [44]:
#Build the model
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [45]:
#Train, evaluate, and predict
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 0s 2ms/step - loss: 0.5021 - accuracy: 0.7337
Epoch 2/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4180 - accuracy: 0.8086
Epoch 3/20
126/126 [==============================] - 0s 1ms/step - loss: 0.4096 - accuracy: 0.8293
Epoch 4/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3910 - accuracy: 0.8389
Epoch 5/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3822 - accuracy: 0.8309
Epoch 6/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3753 - accuracy: 0.8309
Epoch 7/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3710 - accuracy: 0.8357
Epoch 8/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3650 - accuracy: 0.8485
Epoch 9/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3519 - accuracy: 0.8501
Epoch 10/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3457 - accuracy: 0.8533

In [46]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

53/53 [==============================] - 0s 1ms/step - loss: 0.4621 - accuracy: 0.8409


Test Loss 0.4621376693248749, Test Accuracy 0.8409090638160706


In [47]:
#infer labels on a batch or a dataset of batches.
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Predicted survival: 98.07%  | Actual outcome:  SURVIVED
Predicted survival: 72.79%  | Actual outcome:  SURVIVED
Predicted survival: 100.00%  | Actual outcome:  DIED
Predicted survival: 89.26%  | Actual outcome:  DIED
Predicted survival: 5.34%  | Actual outcome:  DIED
